In [1]:
import requests

In [2]:
query = 'sum(increase(http_request_duration_seconds_bucket{path="/ping", code="200"}[10m])) by (le)'
resp = requests.get('http://localhost:9090/api/v1/query', params={'query': query})
data = resp.json()
data

{'data': {'result': [{'metric': {'le': '1'},
    'value': [1511418451.539, '7524.304755189762']},
   {'metric': {'le': '2.5'}, 'value': [1511418451.539, '7567.016040460469']},
   {'metric': {'le': '5'}, 'value': [1511418451.539, '7576.1684587327645']},
   {'metric': {'le': '0.75'}, 'value': [1511418451.539, '7485.661211373406']},
   {'metric': {'le': '0.075'}, 'value': [1511418451.539, '5971.444454990442']},
   {'metric': {'le': '0.25'}, 'value': [1511418451.539, '7124.649157299564']},
   {'metric': {'le': '+Inf'}, 'value': [1511418451.539, '7578.202329459941']},
   {'metric': {'le': '10'}, 'value': [1511418451.539, '7578.202329459941']},
   {'metric': {'le': '0.01'}, 'value': [1511418451.539, '2172.1739366245884']},
   {'metric': {'le': '0.025'},
    'value': [1511418451.539, '3978.2511423573924']},
   {'metric': {'le': '0.1'}, 'value': [1511418451.539, '6338.558121245815']},
   {'metric': {'le': '0.05'}, 'value': [1511418451.539, '5342.978400292878']},
   {'metric': {'le': '0.005'}, 

In [3]:
buckets = sorted([(float(m['metric']['le']), float(m['value'][1])) for m in data['data']['result']])
buckets

[(0.005, 1127.781318219414),
 (0.01, 2172.1739366245884),
 (0.025, 3978.2511423573924),
 (0.05, 5342.978400292878),
 (0.075, 5971.444454990442),
 (0.1, 6338.558121245815),
 (0.25, 7124.649157299564),
 (0.5, 7388.035416468931),
 (0.75, 7485.661211373406),
 (1.0, 7524.304755189762),
 (2.5, 7567.016040460469),
 (5.0, 7576.1684587327645),
 (7.5, 7577.185394096353),
 (10.0, 7578.202329459941),
 (inf, 7578.202329459941)]

In [4]:
bucket_counts = []
for idx, bucket in enumerate(buckets):
    if idx > 0:
        count = bucket[1] - buckets[idx - 1][1]
    else:
        count = bucket[1]
    bucket_counts.append((bucket[0], count))
bucket_counts

[(0.005, 1127.781318219414),
 (0.01, 1044.3926184051743),
 (0.025, 1806.077205732804),
 (0.05, 1364.7272579354858),
 (0.075, 628.4660546975638),
 (0.1, 367.11366625537266),
 (0.25, 786.0910360537491),
 (0.5, 263.3862591693669),
 (0.75, 97.6257949044757),
 (1.0, 38.643543816355304),
 (2.5, 42.711285270707776),
 (5.0, 9.152418272295108),
 (7.5, 1.0169353635883454),
 (10.0, 1.0169353635883454),
 (inf, 0.0)]

In [5]:
max_width = 100
block_value = max(b[1] for b in bucket_counts) / max_width
prev_bound = 0.0
for bound, count in bucket_counts:
    bar = int(count / block_value) * '▇'
    if count > 0 and not bar:
        bar = '▏' # tiny block to indicate non-zero value
    print('{:6.3f} - {:6.3f}: {}'.format(prev_bound, bound, bar))
    prev_bound = bound

 0.000 -  0.005: ▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
 0.005 -  0.010: ▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
 0.010 -  0.025: ▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
 0.025 -  0.050: ▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
 0.050 -  0.075: ▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
 0.075 -  0.100: ▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
 0.100 -  0.250: ▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
 0.250 -  0.500: ▇▇▇▇▇▇▇▇▇▇▇▇▇▇
 0.500 -  0.750: ▇▇▇▇▇
 0.750 -  1.000: ▇▇
 1.000 -  2.500: ▇▇
 2.500 -  5.000: ▏
 5.000 -  7.500: ▏
 7.500 - 10.000: ▏
10.000 -    inf: 
